In [57]:
from h5py import File
import pandas as pd
from src.utils.progbar import ProgBar
from ase.neighborlist import PrimitiveNeighborList 
import numpy as np

In [1]:
# with File('data/OCH.h5', 'r') as f:
#     cols = [col for col in list(f.values())[0].keys() if col != "subset"]
#     inds = list(f.keys())
#     df = pd.DataFrame(columns=cols, index=inds)
#     for ind in inds:
#         for col in cols:
#             try:
#                 df.loc[ind][col] = f[f"{ind}/{col}"].value
#             except KeyError:
#                 pass
# df.head(5)

In [2]:
one_cols = ['atomic_numbers', 'smiles'] # Columns consistent across all conformations
chunk_size = 1000 # All conformations for 20 molecules
current_chunk = 0
with File('data/OCH.h5', 'r') as f:
    cols = [col for col in list(f.values())[0].keys() if col != "subset"]
    inds = [] # Indices for each row, corresponding to a molecule index and conformation number
    mols = list(f.keys()) # Indices for individual molecules
    for mol in mols:
        conformations = f[f'{mol}/conformations'].shape[0]
        for i in range(conformations):
            inds.append(f"{mol}/{i}")
    n_chunks = len(inds) // chunk_size #! This might exclude some data due to rounding down, need to check
    prg = ProgBar(n_chunks)
    while True:
        prg.update(current_chunk)
        if current_chunk >= len(inds) // chunk_size:
            break
        _inds = inds[current_chunk*chunk_size:(current_chunk+1)*chunk_size]    
        df = pd.DataFrame(columns=cols, index=_inds)
        for _ind in _inds:
            for col in cols:
                # Case where column has distinct value for each conformation
                try:
                    if col not in one_cols:
                        mol, i = _ind.split("/")
                        df.loc[_ind][col] = f[f"{mol}/{col}"][()][int(i)]
                    else: 
                        df.loc[_ind][col] = f[f"{mol}/{col}"][()]
                except KeyError:
                    pass
        df.to_hdf("data/OCH_ext.h5", key=f"chunk_{current_chunk}")
        current_chunk+=1


/home/jonathanwright/miniconda3/envs/ml_mp_env/lib/python3.6/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['atomic_numbers', 'conformations', 'dft_total_energy',
       'dft_total_gradient', 'formation_energy', 'mayer_indices',
       'mbis_charges', 'mbis_dipoles', 'mbis_octupoles', 'mbis_quadrupoles',
       'scf_dipole', 'scf_quadrupole', 'smiles', 'wiberg_lowdin_indices'],
      dtype='object')]

  encoding=encoding,


In [4]:
df = pd.read_hdf('data/processed/OCH_ext_test1.h5', "chunk_0", 'r')

In [11]:
# df.conformations[0]
row1 = df.iloc[0]
# df.atomic_numbers[0]
row1

atomic_numbers           [8, 8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, 6, 6, 6, ...
conformations            [[-3.7761912, 0.2544896, -7.6026983], [1.53757...
dft_total_energy                                                  -1142.93
dft_total_gradient       [[-0.03294501, -0.0059776297, -0.031587306], [...
formation_energy                                                  -6.19246
mayer_indices            [[0.0, 0.05211287, 5.511885e-05, 3.868582e-05,...
mbis_charges             [[-0.42718434], [-0.39027286], [-0.5742869], [...
mbis_dipoles             [[-0.039469335, 0.006042725, -0.051155943], [0...
mbis_octupoles           [[[[0.15195373 0.03321473 0.0536355 ], [ 0.033...
mbis_quadrupoles         [[[-4.3716974, 0.047171943, 0.027945584], [0.0...
scf_dipole                            [-1.3032234, -0.38430625, 2.0021617]
scf_quadrupole           [[-96.58924, 1.2722458, -9.466976], [1.2722458...
smiles                   [[H:33][c:19]1[c:18]([c:20]([c:21]([c:22]2[c:2...
wiberg_lowdin_indices    

In [55]:
atom_num_map = {8:"O", 6: "C", 1:"H"}

def interatomic_distances(row):
    """ 
    Calculate the interatomic distances for a given conformation
    Annotate based on the two atoms involved
    """
    print(row.conformations[0])
    print(row.conformations[1])

    return np.linalg.norm([
        [np.array(row.conformations[0]), (row.conformations[1])],
        [np.array(row.conformations[0]), (row.conformations[1])]
    ],axis=(1,2))

In [97]:
from ase.neighborlist import neighbor_list
from ase import Atoms

a = Atoms(
    numbers=row1.atomic_numbers,
    positions=row1.conformations
)

b = neighbor_list('ijd', a, cutoff=2.5, self_interaction=False)


In [98]:
b

(array([ 0,  1,  2,  2,  3,  4,  5,  7,  8,  9, 10, 11, 13, 15, 17, 18, 19,
        20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]),
 array([ 7,  8, 23,  9, 24, 25, 26,  0,  1,  2, 27, 28, 29, 30, 31, 32, 20,
        19,  2,  3,  4,  5, 10, 11, 13, 15, 17, 18]),
 array([2.34690532, 2.34801044, 1.86808924, 2.49766506, 1.7865955 ,
        1.83831925, 2.00485411, 2.34690532, 2.34801044, 2.49766506,
        2.10653579, 1.98753989, 2.08968685, 2.01348883, 2.0877547 ,
        2.1060767 , 2.47298836, 2.47298836, 1.86808924, 1.7865955 ,
        1.83831925, 2.00485411, 2.10653579, 1.98753989, 2.08968685,
        2.01348883, 2.0877547 , 2.1060767 ]))

In [93]:
# b.get_neighbors([i for i in range(a.numbers.shape[0])])
b.neighbor_list(0)

AttributeError: 'NeighborList' object has no attribute 'neighbor_list'